In [15]:
import sys
if '..' not in sys.path:
    sys.path.append('..')

import torch
from src.model import CRNN
from src.utils import preprocess_imagenet, get_code

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [35]:
model_path = '../weights/model_ocr.pth'

In [36]:
model = CRNN(    
    cnn_backbone_name = 'resnet18d',
    cnn_backbone_pretrained = True,
    cnn_output_size = 4608,
    rnn_features_num = 128,
    rnn_dropout = 0.1,
    rnn_bidirectional = True,
    rnn_num_layers = 2,
    num_classes = 12,
)

In [38]:
state_dict = torch.load(model_path, map_location=torch.device('cpu'))['my_state_dict']
model.load_state_dict(state_dict, strict=False)
model.eval()

## Convert to script

In [55]:
VOCAB = ' 0123456789'
DEVICE = 'cpu'
IMG_SIZE = (512, 280)

In [54]:
class ModelWrapper(torch.nn.Module):
    def __init__(self, model, vocab, size):
        super().__init__()
        self.model = model
        self.vocab = vocab
        self.size = size
    
    def forward(self, image):
        return self.model.forward(image)

In [56]:
wrapper = ModelWrapper(model, vocab=VOCAB, size=IMG_SIZE)

In [57]:
scripted_model = torch.jit.script(wrapper)

In [58]:
torch.jit.save(scripted_model, model_path.replace('.pth', '.zip'))